In [1]:
!export PYTHONPATH=.:$PYTHONPATH

In [2]:
from odi.feature_engg import feature_extractor as fe
from odi.feature_engg import util as cricutil
from odi.preprocessing import rank
from odi.retrain import create_train_test_ci as ctt
from odi.model_util import odi_util as outil
import pandas as pd
from datetime import datetime
import math
import numpy as np
from tqdm.notebook import tqdm
import os

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline


from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,accuracy_score

In [3]:
import pickle

In [85]:
train_x=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.mg_train_x),'rb'))
train_y=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.mg_train_y),'rb'))

test_x=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.mg_test_x),'rb'))
test_y=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.mg_test_y),'rb'))

#cols = pickle.load(open(os.path.join(outil.DEV_DIR, ctt.second_innings_base_columns), 'rb'))

In [86]:
def set_loc(row):
    if row['team_a_loc']==1:
        return 0
    elif row['team_b_loc']==1:
        return 1
    else: 
        return 2

In [87]:
train_x_df = pd.DataFrame(train_x)
train_x_df.columns =['strngth','toss','team_a_loc','team_b_loc','other_loc']

test_x_df = pd.DataFrame(test_x)
test_x_df.columns =['strngth','toss','team_a_loc','team_b_loc','other_loc']

train_x_df['loc'] = train_x_df.apply(set_loc,axis=1)
test_x_df['loc'] = test_x_df.apply(set_loc,axis=1)

train_x = np.array(train_x_df[['strngth','toss','loc']])
test_x = np.array(test_x_df[['strngth','toss','loc']])



In [88]:
train_x.shape,train_y.shape

((288, 3), (288,))

In [89]:
test_x.shape

(89, 3)

In [90]:
# from sklearn.feature_selection import SequentialFeatureSelector
# from sklearn.linear_model import LinearRegression,LogisticRegression
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

In [91]:
# pipe = Pipeline([('scaler', StandardScaler()), ('regression', LinearRegression())])
# sfs = SequentialFeatureSelector(pipe, n_features_to_select=3)
# sfs.fit(train_x, train_y)

In [92]:
#pd.DataFrame(train_x)

In [110]:
from sklearn.ensemble import GradientBoostingClassifier

train_pipe = Pipeline([('polynom',PolynomialFeatures(1)), ('cls', LogisticRegression())])
#train_pipe = Pipeline([('polynom',PolynomialFeatures(1)), ('cls', GaussianNB())])
#train_pipe = Pipeline([('polynom',PolynomialFeatures(1)), ('svc',SVC())])
#train_pipe = Pipeline([('polynom',PolynomialFeatures(1)), ('GBC', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0))])



In [111]:
train_pipe.fit(train_x,train_y)

Pipeline(steps=[('polynom', PolynomialFeatures(degree=1)),
                ('cls', LogisticRegression())])

In [112]:
train_predict = train_pipe.predict(train_x)
test_predict = train_pipe.predict(test_x)

In [113]:
accuracy_score(train_y,train_predict)



0.6006944444444444

In [114]:
accuracy_score(test_y,test_predict)

0.5955056179775281

In [115]:
accuracy_score(test_y[:45],test_predict[:45])

0.6666666666666666